In [49]:
# Main Libraries
import pandas as pd 
import numpy as np
from numpy import *

#Nice graphing tools
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import plotly
import plotly.offline as py
import plotly.tools as tls
import plotly.graph_objs as go
import plotly.tools as tls

from multiprocessing import Pool
from itertools import product
import knntsp
import warnings
import importlib
warnings.filterwarnings('ignore')

import pickle

NUM_CORES = 4
PREPROCESSED_PATH = './../preprocessed/'
RESULTS_PATH = './../results/'

In [33]:
# Reading groupped data
def read_groupped_data(filename):
    timeSeries = pd.read_csv(filename,index_col=['Unnamed: 0'], parse_dates=['Unnamed: 0'])
    timeSeries.rename(columns=lambda x: eval(x), inplace=True)
    timeSeries.index = pd.to_datetime(timeSeries.index)
    return timeSeries

In [40]:
NUM_CORES = 4
RANDOM_STATE = 1
TRAIN_SIZE = 0.86

In [35]:
# Reading time series data
timeSeries = read_groupped_data(PREPROCESSED_PATH + 'timeseries.csv')

In [36]:
# Removing time series without data
timeSeries = timeSeries.dropna(axis=1)

In [37]:
cluster_labels = pd.read_csv(PREPROCESSED_PATH + 'normalization_based_cluster_labels.csv', header=-1, index_col=[0])
cluster_labels.columns = ['label']
cluster_labels.head()

simple_w_k = pd.read_csv(PREPROCESSED_PATH + 'simple_knntsp_w_k.csv')
cluster_w_k = pd.read_csv(PREPROCESSED_PATH + 'cluster_knntsp_w_k.csv')

In [46]:
importlib.reload(knntsp)

results_cluster_knn_real = []
results_cluster_knn_predicted = []

results_simple_knn_real = []
results_simple_knn_predicted = []

to_run_cluster_real = []
to_run_cluster_predicted = []

to_run_simple_real = []
to_run_simple_predicted = []

X = timeSeries.copy()
_cluster_labels = cluster_labels.copy()

for column in X.columns:
    label = cluster_labels.iloc[X.T.index.get_loc(column)][0]
    
    w_k = simple_w_k
    clust_w_k = cluster_w_k[cluster_w_k.cluster_label==label]
    if len(clust_w_k) == 0:
        clust_w_k = simple_w_k
    
    to_run_cluster_real.append([X, column, _cluster_labels, int(clust_w_k.iloc[0]['w']), int(clust_w_k.iloc[0]['k']), TRAIN_SIZE, True])
    to_run_cluster_predicted.append([X, column, _cluster_labels, int(clust_w_k.iloc[0]['w']), int(clust_w_k.iloc[0]['k']), TRAIN_SIZE, False])
    
    to_run_simple_predicted.append([X, column, int(simple_w_k['w'][0]), int(simple_w_k['k'][0]), TRAIN_SIZE, False, label, False])
    to_run_simple_real.append([X, column, int(simple_w_k['w'][0]), int(simple_w_k['k'][0]), TRAIN_SIZE, False, label, True])

with Pool(NUM_CORES) as pool:
    results_cluster_knn_real = pool.starmap(knntsp.predict_with_cluster_knn, to_run_cluster_real)
    results_simple_knn_real = pool.starmap(knntsp.predict_with_knn, to_run_simple_real)
    results_cluster_knn_predicted = pool.starmap(knntsp.predict_with_cluster_knn, to_run_cluster_predicted)
    results_simple_knn_predicted = pool.starmap(knntsp.predict_with_knn, to_run_simple_predicted)

In [50]:
SAVE_PICKLE_RESULTS = False

if SAVE_PICKLE_RESULTS:
    output = open(RESULTS_PATH + 'results_cluster_knn_real.pk', 'wb')
    pickle.dump(results_cluster_knn_real, output)
    output.close()
    
    output = open(RESULTS_PATH + 'results_simple_knn_real.pk', 'wb')
    pickle.dump(results_simple_knn_real, output)
    output.close()
    
    output = open(RESULTS_PATH + 'results_cluster_knn_predicted.pk', 'wb')
    pickle.dump(results_cluster_knn_predicted, output)
    output.close()
    
    output = open(RESULTS_PATH + 'results_simple_knn_predicted.pk', 'wb')
    pickle.dump(results_simple_knn_predicted, output)
    output.close()   